# Crypto Quantitative Analysis & Historical Data Management

This notebook demonstrates how to:
1. **Set up an interactive analysis environment** for crypto time-series data
2. **Access current OHLCV data** from our TimescaleDB instance
3. **Implement efficient historical data storage** strategies
4. **Create time bucketing and continuous aggregations** for fast queries
5. **Build scalable data pipelines** for multi-year historical data

## 🎯 Goals
- Understand where our current data is stored and how to access it
- Implement best practices for storing 1-minute data from 2025-01-01 to now
- Set up TimescaleDB partitioning and continuous aggregations
- Create efficient querying patterns for technical analysis and ML

# 1. Set Up Jupyter Notebook Environment

First, let's configure our notebook environment with the necessary settings for optimal crypto data analysis.

In [ ]:
# Configure notebook environment
import warnings
warnings.filterwarnings('ignore')

# Set display options for better data visualization
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 100)

# Configure plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Notebook environment configured successfully!")
print("📊 Pandas display options optimized for time-series data")
print("📈 Matplotlib style set to seaborn for better visualizations")

# 2. Import Essential Libraries

Import all necessary libraries for data analysis, database connections, visualization, and technical indicators.

In [ ]:
# Core data analysis libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from typing import List, Optional, Dict

# Database connections
import psycopg  # PostgreSQL/TimescaleDB connector
from sqlalchemy import create_engine, text

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Technical analysis
try:
    import ta  # Technical Analysis library
    print("✅ Technical Analysis library (ta) available")
except ImportError:
    print("⚠️ Installing technical analysis library...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'ta'])
    import ta

# Machine Learning (optional)
try:
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    from sklearn.model_selection import train_test_split
    print("✅ Scikit-learn available for ML preprocessing")
except ImportError:
    print("ℹ️ Scikit-learn not available (install if needed for ML)")

print("📚 All essential libraries imported successfully!")

# Database configuration
DB_CONFIG = {
    'host': 'localhost',
    'port': 5433,
    'database': 'quant_db',
    'user': 'quant_user',
    'password': 'quant_password'
}

DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
print(f"🔗 Database URL configured: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

# 3. Load and Inspect Current Data

Let's examine what data we currently have in our TimescaleDB instance and understand its structure.

In [ ]:
def get_data_summary():
    """Get summary of current data in TimescaleDB"""
    try:
        with psycopg.connect(DATABASE_URL) as conn:
            # Data summary query
            summary_query = """
            SELECT 
                symbol,
                COUNT(*) as total_records,
                MIN(time) as earliest_time,
                MAX(time) as latest_time,
                ROUND(AVG(volume)::numeric, 2) as avg_volume,
                ROUND(AVG(close)::numeric, 2) as avg_price,
                ROUND((MAX(time) - MIN(time))::numeric / 3600, 2) as hours_of_data
            FROM ohlcv_1min 
            GROUP BY symbol 
            ORDER BY total_records DESC;
            """
            
            df_summary = pd.read_sql_query(summary_query, conn)
            
            # Overall statistics
            total_query = """
            SELECT 
                COUNT(*) as total_records,
                COUNT(DISTINCT symbol) as unique_symbols,
                MIN(time) as earliest_time,
                MAX(time) as latest_time
            FROM ohlcv_1min;
            """
            
            overall_stats = pd.read_sql_query(total_query, conn)
            
            return df_summary, overall_stats
            
    except Exception as e:
        print(f"❌ Database connection error: {e}")
        return None, None

# Get current data summary
summary_df, overall_stats = get_data_summary()

if summary_df is not None:
    print("📊 CURRENT DATA SUMMARY")
    print("=" * 50)
    print(f"🔢 Total Records: {overall_stats['total_records'].iloc[0]:,}")
    print(f"📈 Unique Symbols: {overall_stats['unique_symbols'].iloc[0]}")
    print(f"⏰ Time Range: {overall_stats['earliest_time'].iloc[0]} to {overall_stats['latest_time'].iloc[0]}")
    
    duration = overall_stats['latest_time'].iloc[0] - overall_stats['earliest_time'].iloc[0]
    hours = duration.total_seconds() / 3600
    print(f"⏱️ Total Duration: {hours:.1f} hours")
    
    print("\n📋 PER-SYMBOL BREAKDOWN:")
    print(summary_df.to_string(index=False))
else:
    print("❌ Could not connect to database or retrieve data")